Wifi Networks
===================================================

Cleaning up list of devices present at ITU for the entire period


In [1]:
from lib.cached_data_loader import CachedDataLoader
from lib.panda_plotter import PandaPlotter
cdl = CachedDataLoader()
pp = PandaPlotter()


In [3]:
# COUNT DISTING DEVICES
sql = "SELECT COUNT(DISTINCT(wc.client_mac)) AS DEVICE_COUNT FROM wifi_clients AS wc;"
df = cdl.get_dataframe(sql)
print(df)

   DEVICE_COUNT
0          9113


In [4]:
# COUNT DISTING USERS
sql = "SELECT COUNT(DISTINCT(wc.user_name)) AS USER_COUNT FROM wifi_clients AS wc;"
df = cdl.get_dataframe(sql)
print(df)


   USER_COUNT
0        7267
